
Nombre: Alejandro Velazco 
Busca y descarga el conjunto de datos “Climate Model Simulation
Crashes” del repositorio UCI Machine Learning. Tiene los datos de un
problema de clasificación.
2. Cárgalo adecuadamente en un Pandas DataFrame.
3. Realiza un conteo de las clases y analiza si hay desequilibrio.
4. Evalua el rendimiento del modelo k-Nearest Neigbours mediante las
tres estrategias de validación cruzada estudiados y utilizando la
métrica que consideres más adecuada.

In [2]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold, LeaveOneOut, train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler

ModuleNotFoundError: No module named 'sklearn'

In [ ]:

# 1. Descargar el conjunto de datos
url = "./pop_failures.dat"

data = pd.read_csv(url, delimiter=r"\s+")  # El archivo tiene 21 líneas de metadatos al principio

# 2. Cargarlo en un Pandas DataFrame
df = data.copy()

# 3. Conteo de clases y análisis de desequilibrio
print(df['outcome'].value_counts())  # 'outcome' es la columna de etiquetas

# 4. Evaluar el rendimiento de k-Nearest Neighbors

# Separar las características y etiquetas
X = df.drop(['outcome'], axis=1).values
y = df['outcome'].values

# Normalizar los datos
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Crear modelo KNN
knn = KNeighborsClassifier(n_neighbors=5)

# Estrategias de validación cruzada
kf = KFold(n_splits=10)
loocv = LeaveOneOut()

# Validación cruzada Monte Carlo
monte_carlo_scores = []

for _ in range(100):  # Realizar 100 iteraciones/splits aleatorios
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3)
    knn.fit(X_train, y_train)
    monte_carlo_scores.append(knn.score(X_test, y_test))
monte_carlo_accuracy = np.mean(monte_carlo_scores)

print(f"KFold Accuracy: {np.mean(cross_val_score(knn, X_scaled, y, cv=kf)):.4f}")
print(f"LOOCV Accuracy: {np.mean(cross_val_score(knn, X_scaled, y, cv=loocv)):.4f}")
print(f"Monte Carlo Accuracy: {monte_carlo_accuracy:.4f}")

NameError: name 'pd' is not defined